In [2]:
q='export PATH=/homes/lgaiser/miniconda3/envs/q2dev-amplicon/bin:$PATH'

In [3]:
! $q;  qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


# Preprocessing

<font size="2"> <p> Importing reference hits from the first sequencing run </p> </font>

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/From_DADA2/first_run/feature_table_first_run_transposed.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_first_run.qza

In [ ]:
! $q; qiime tools import \
    --input-path EDENISS/From_DADA2/first_run/rep_seqs_first_run.fasta \
    --output-path EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run.qza \
    --type 'FeatureData[Sequence]'

<font size="2"> <p> Importing reference hits from the missing samples in a second sequencing run </p> </font>

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/From_DADA2/second_run/feature_table_second_run_transposed.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_second_run.qza

In [ ]:
! $q; qiime tools import \
    --input-path EDENISS/From_DADA2/second_run/rep_seqs_second_run.fasta \
    --output-path EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run.qza \
    --type 'FeatureData[Sequence]'

Import taxonomic annotation from DADA2 

In [ ]:
! $q; qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path EDENISS/From_DADA2/first_run/taxonomy_first_run.tsv \
  --output-path EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza \
  --input-format HeaderlessTSVTaxonomyFormat

In [ ]:
! $q; qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path EDENISS/From_DADA2/second_run/taxonomy_second_run.tsv \
  --output-path EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
  --input-format HeaderlessTSVTaxonomyFormat

<font size="2"> <p> Using the taxonomic classification from the previous step to remove reads stemming from mitochondria and chloroplasts from the feature table of the first sequencing run </p> </font>

In [ ]:
! $q; qiime taxa filter-table \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run.qza \
    --i-taxonomy  EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza \
    --p-exclude mitochondria,chloroplast \
    --verbose \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza

<font size="2"> <p> Filtering the feature table from the second run in the same way </p> </font>

In [ ]:
! $q; qiime taxa filter-table \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run.qza \
    --i-taxonomy  EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza

Both taxonomies were then merged for later use

In [ ]:
! $q;  qiime feature-table merge-taxa \
    --i-data EDENISS/Analysis_Paper_DADA2/taxonomy_first_run.qza EDENISS/Analysis_Paper_DADA2/taxonomy_second_run.qza \
    --o-merged-data EDENISS/Analysis_Paper_DADA2/taxonomy-final.qza

# Quality control

The q2-quality-control plugin was used to remove contaminants introduced during DNA extraction and library preparation.

decontam-identify was used to calculate the likelihood that a read was the result of a contamination

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-blank \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza 

decontam-score-viz was used to visualize the likelihood scores in a histogram to decide on a cutoff value for reads to be discarded

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-first-run.qzv

decontam-remove was used to filter out reads with values below the chosen threshold

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-first-run.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run-filtered.qza \
    --p-threshold 0.1 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run_filterd_seqs_decontam.qza

The same process was used to filter the samples from the second run

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_FEG9_FEG10_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-blank \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-second-run.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-second-run.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run.qza \
    --p-threshold 0.4 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run_filterd_seqs_decontam.qza

The NTC and ZymoStandard columns were removed from both feature tables. Feature tables and repseqs from both runs were then merged 

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/Analysis_Paper_DADA2/import/feature_table_first_run_decontam_noctrl.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam_noctrl.qza

In [ ]:
! $q; qiime tools import \
  --type 'FeatureTable[Frequency]' \
  --input-path EDENISS/Analysis_Paper_DADA2/import/feature_table_second_run_decontam_noctrl.biom \
  --output-path EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam_noctrl.qza

In [ ]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature_table_first_run_decontam_noctrl.qza EDENISS/Analysis_Paper_DADA2/feature_table_second_run_decontam_noctrl.qza \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-combined.qza

In [ ]:
! $q; qiime feature-table merge-seqs \
    --i-data EDENISS/Analysis_Paper_DADA2/rep_seqs_first_run_filterd_seqs_decontam.qza EDENISS/Analysis_Paper_DADA2/rep_seqs_second_run_filterd_seqs_decontam.qza \
    --o-merged-data EDENISS/Analysis_Paper_DADA2/rep-seqs-combined.qza

After merging the two sequencing runs the table was split into tables for pre-cleaning, post-cleaning and samples from the second campaign

In [ ]:
! $q; qiime feature-table split \
    --i-table EDENISS/Analysis_Paper_DADA2/feature-table-combined.qza \
    --m-metadata-file EDENISS/Metadata/Metadata_combined_DADA2.tsv \
    --m-metadata-column split_table \
    --o-tables EDENISS/Analysis_Paper_DADA2/split_tables

The pre- and post-cleaning samples were again filtered with decontam to remove contaminations that were caused during the sample aqcuisition. This step was skipped for the samples from the second campaign due to the lack of control samples.

Pre-cleaning

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_pre_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-sampling_pre \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-cleaning.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-cleaning.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-pre-cleaning.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-pre-filtered.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Pre-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-combined-filtered.qza \
    --p-threshold 0.14 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature-table-pre-cleaning-filtered.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-pre-cleaning-filtered.qza

Post-cleaning

In [ ]:
! $q; qiime quality-control decontam-identify \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post.qza \
    --m-metadata-file EDENISS/Metadata/decontam_metadata_post_DADA2.tsv \
    --p-method combined \
    --p-freq-concentration-column DNA-concentration \
    --p-prev-control-column sample-type \
    --p-prev-control-indicator control-sampling_post \
    --o-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-cleaning.qza 

In [ ]:
! $q; qiime quality-control decontam-score-viz \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-cleaning.qza   \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post.qza \
    --o-visualization EDENISS/Analysis_Paper_DADA2/decontam/hist-post-cleaning.qzv

In [ ]:
! $q; qiime quality-control decontam-remove \
    --i-decontam-scores EDENISS/Analysis_Paper_DADA2/decontam/scores-post-filtered.qza \
    --i-table EDENISS/Analysis_Paper_DADA2/split_tables/Post-filtered.qza \
    --i-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-combined.qza \
    --p-threshold 0.18 \
    --o-filtered-table EDENISS/Analysis_Paper_DADA2/feature-table-post-cleaning-filtered.qza \
    --o-filtered-rep-seqs EDENISS/Analysis_Paper_DADA2/rep-seqs-post-cleaning-filtered.qza

After filtering the pre- and post-cleaning feature tables were merged again

In [ ]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature-table-pre-cleaning-filtered.qza EDENISS/Analysis_Paper_DADA2/feature-table-post-cleaning-filtered.qza  \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza

In [4]:
! $q; qiime feature-table merge \
    --i-tables EDENISS/Analysis_Paper_DADA2/feature-table-combined-filtered.qza EDENISS/Analysis_Paper_DADA2/split_tables/FEG9.qza  \
    --o-merged-table EDENISS/Analysis_Paper_DADA2/feature-table-all-filtered.qza

Saved FeatureTable[Frequency] to: EDENISS/Analysis_Paper_DADA2/feature-table-all-filtered.qza
